**Проектная работа: 
Рынок заведений общественного питания Москвы.**

**Описание проекта:**
Инвесторы из фонда «Shut Up and Take My Money» решили попробовать себя в новой области и открыть заведение общественного питания в Москве. Заказчики ещё не знают, что это будет за место: кафе, ресторан, пиццерия, паб или бар, — и какими будут расположение, меню и цены.
Нам нужно подготовить исследование рынка Москвы, найти интересные особенности и презентовать полученные результаты, которые в будущем помогут в выборе подходящего инвесторам места.

**Шаг 1. Загрузим данные и изучим общую информацию**

In [1]:
# импорт библиотек
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings; warnings.filterwarnings(action='ignore')
from plotly import graph_objects as go
from IPython.display import set_matplotlib_formats
import json
import folium
# импортируем карту и маркер
from folium import Map, Choropleth, Marker
# импортируем кластер
from folium.plugins import MarkerCluster
import re

ModuleNotFoundError: No module named 'folium'

In [ ]:
moscow_places = pd.read_csv('https://code.s3.yandex.net/datasets/moscow_places.csv')

In [ ]:
moscow_places.info()
moscow_places.head(10)

In [ ]:
plt.hist(moscow_places['rating'], bins=20)
plt.title('Рейтинг заведений общественного питания')
plt.xlabel('Рейтинг')
plt.ylabel('Количество заведений')
plt.show()

In [ ]:
print(moscow_places['seats'].describe())
moscow_places.query('seats > 1000').sample(5)

Данные загружены. Всего у нас 8406 заведений. В среднем 108 посадочных мест, а медиана 75, но есть и заведения с 1000+ местами, скорее всего для банкетов. Большентсво заведений имеют рейтинг 4 и больше.

**Шаг 2. Выполните предобработку данных**

In [ ]:
#проверим на дуликаты
moscow_places.duplicated().sum()

ДУбликатов нет. 

In [ ]:
#проверим на неявные дубликаты
duplicates = moscow_places.duplicated(subset=['name', 'address'], keep=False)
if duplicates.any():
    duplicate_rows = moscow_places[duplicates]
    print("Обнаружены неявные дубликаты:")
    print(duplicate_rows)
else:
    print("Неявные дубликаты не обнаружены.")

In [ ]:
#проверим есть ли пропуски в данных 
moscow_places.isna().sum()

Пропуски пристуствуют в 6 столбцах. Я считаю что их следует оставить для коректности данных, пропусков много и их ничем не заменить. 

In [ ]:
# замена типа данных в столбце chain на булев.
moscow_places['chain'] = moscow_places['chain'].astype(bool)

Создадим столбец street с названиями улиц из столбца с адресом.

In [ ]:
moscow_places['street'] = moscow_places['address'].str.split(',', expand=True)[1]
moscow_places['street'] = moscow_places['street'].str.strip()

Создадим столбец is_24/7 с обозначением, что заведение работает ежедневно и круглосуточно (24/7):
- логическое значение True — если заведение работает ежедневно и круглосуточно;
- логическое значение False — в противоположном случае.

In [ ]:
# создание столбца is_24/7
moscow_places['is_24/7'] = moscow_places['hours'].str.contains('ежедневно', case=False) & moscow_places['hours'].str.contains('круглосуточно', case=False)

# преобразование значений в логические
moscow_places['is_24/7'] = moscow_places['is_24/7'].astype(bool)
moscow_places.columns

Предобработка данных выполнена: дубликатов нет, пропуски оставила, создала 2 новых столбца. 

**Шаг 3. Анализ данных**

**Исследуем количество объектов общественного питания по категориям: рестораны, кофейни, пиццерии, бары и так далее.**

In [ ]:
category_moscow_places = moscow_places.groupby('category').agg(count=('name','count')).reset_index().sort_values('count', ascending=False)
category_moscow_places['percent'] = category_moscow_places['count'] / sum(category_moscow_places['count']) * 100
category_moscow_places

In [ ]:
# группируем данные по категориям и находим топ-15 популярных
categories = moscow_places.groupby('category')['name'].count().sort_values(ascending=False)

# создаем график
plt.figure(figsize=(10,6))
sns.barplot(x=categories, y=categories.index, palette='bright')
plt.title('Количество заведений общественного питания по категориям')
plt.xlabel('Количество заведений')
plt.ylabel('Категория заведени')
plt.show()

Самая большая категория это - кафе(28.28%). 2-ое место рестораны(24.30) и 3-е кофейни(16,80). Всё вместе примерно 69,38% от общего количества. 

**Исследуем количество посадочных мест в местах по категориям: рестораны, кофейни, пиццерии, бары и так далее.**

In [ ]:
plt.figure(figsize=(10, 6))
sns.boxplot(x='category', y='seats', data=moscow_places, showfliers=False, palette='bright')
plt.title('Количества посадочных мест в зависимости от типа заведения')
plt.xlabel('Категория заведения')
plt.ylabel('Количество посадочных мест')
plt.xticks(rotation=45)
plt.show()

У рестаранов, баров, кофейни и быстрого питания максимальное число посадочных мест достигает 300-а, а в среднем 150. У остальных категорий максимальное кол-во мест доходит до 150-и, а всреднем до 125.

**Рассмотрим соотношение сетевых и несетевых заведений в датасете.**

In [ ]:
chain_info = moscow_places.groupby(['category', 'chain'])['name'].count().reset_index()
independent = chain_info[chain_info['chain'] == False]
chain = chain_info[chain_info['chain'] == True]
fig, ax = plt.subplots(figsize=(8, 6))
ax.pie(
    [independent['name'].sum(), chain['name'].sum()],
    labels=['Несетевые', 'Сетевые'],
    autopct='%1.1f%%',
    explode=[0.05, 0],
    shadow=True,
    startangle=90,
    colors=['#3A0CA3', '#F72585'],
)
ax.set_title('Соотношение сетевых и несетевых заведений')
plt.show()

Получается что 61,9% несететвые и 38,1% сетевые заведения. 

**Выясним какие категории заведений чаще являются сетевыми.** 

In [ ]:
moscow_places_chains = moscow_places.groupby(['category', 'chain'])['name'].count().reset_index()
moscow_places_chains.columns = ['type', 'chain', 'count']
moscow_places_chains['chain'] = moscow_places_chains['chain'].astype(object)
moscow_places_chains = moscow_places_chains.sort_values(['count', 'chain'])
moscow_places_chains

fig = px.bar(moscow_places_chains,
             x='count',
             y='type',
             text= 'count',
             color='chain',
             category_orders={"chain":['сетевой', 'несетевой']},
             color_discrete_sequence=["#E69F00", "#56B4E9"],
             height=500,
             width=700
             )
fig.update_layout(title = 'Соотношение сетевых и несетевых заведений',
                  xaxis_title = 'Количество заведений',
                  yaxis_title = 'Категории')
fig.show()

In [ ]:
#Код ревьюера
df_chain_share = moscow_places.pivot_table(index='category', columns='chain', values='name', aggfunc='count')
df_chain_share.columns = ['not_chain', 'chain']
df_chain_share['total'] = df_chain_share['not_chain'] + df_chain_share['chain']
df_chain_share['share'] = (df_chain_share['chain']/df_chain_share['total']*100).round().astype('int')
df_chain_share.sort_values(by='share', ascending=False)

In [ ]:
df_chain_share.reset_index(inplace=True)
df_chain_share = df_chain_share.sort_values(by='share', ascending=False)
df_chain_share

In [ ]:
fig = px.bar(df_chain_share
             ,x='chain'
             ,y='share'
             ,color='category'
             ,template='gridon'
             ,text='share',
             labels={'category':'Кол-во объектов:'}
            )
fig.update_xaxes(tickangle=25)
fig.update_layout(
    title='Доля сетевых объектов от всех объектов общественного питания (по типам)',
    xaxis_title="Кол-во заведений, шт",
    yaxis_title="% заведений")
fig.update_traces(textposition='outside')

for trace, percent in zip(fig.data, df_chain_share['chain'].astype('str')):
    trace.name = trace.name.split('=')[-1] + ' (' + percent+ ' объектов)'
    
fig.show()

Самая большая доля сетевых заведений это булочная 61%, пиццерия 52% и кофейня 51%. 

In [ ]:
#Код ревьюера
plt.figure(figsize=(12, 6))
ax = sns.barplot(x='share', y='category', data=df_chain_share.reset_index().sort_values(by='share', ascending=False))
ax.set_xlabel('Процент сетевых заведений')
ax.set_ylabel('Категория')
ax.set_title('Доля сетевых заведений по категориям')

plt.grid()
plt.show()

**Сгруппируем данные по названиям заведений и найдите топ-15 популярных сетей в Москве.**

In [ ]:
#группируем топ 15 заведений Москвы 
top_chains = moscow_places[moscow_places['chain'] == True].groupby('name').count().sort_values(by='category', ascending=False).head(15)

# создаем график
plt.figure(figsize=(10,10))
sns.barplot(x='category', y=top_chains.index, data=top_chains, palette='bright')
plt.title('Топ-15 популярных сетей в Москве')
plt.xlabel('Количество заведений')
plt.ylabel('Название сети')
plt.xticks(rotation=45)
plt.show()

Самые популярные сети это Шоколадница, Домино'с пицца и ДоДо пицца. Большинство заведений имеют глобальное присутствие и имеют строгий брендинг в своих заведениях.

In [ ]:
# группируем данные по категориям и находим топ-15 популярных
top_chains = moscow_places[moscow_places['chain'] == True].groupby('category')['name'].count().sort_values(ascending=False)

# создаем график
plt.figure(figsize=(10,6))
sns.barplot(x=top_chains, y=top_chains.index, palette='bright')
plt.title('Топ категорий сетевых заведений в Москве')
plt.xlabel('Количество заведений')
plt.ylabel('Категория заведени')
plt.show()

По категориям сетевых мест выходит, что кафе, рестораны и кофейни, занимают большую часть сетевых заведений. В категории пиццерия, есть два больших игрока "Доминос Пицца" и "Додо Пицца", которые занимают большую часть категории пиццерия среди сетевых мест.

**Узнаем какие административные районы Москвы присутствуют в датасете. Отобразим общее количество заведений и количество заведений каждой категории по районам.**

In [ ]:
# узнаем какиe районы в Mоскве присутствуют у нас в данных
moscow_places['district'].unique()

In [ ]:
moscow_district = moscow_places.groupby(['category', 'district'])['name'].count().reset_index()
# создаем график
plt.figure(figsize=(15,10))
sns.barplot(x='district', y='name', hue='category', data=moscow_district)
plt.title('Административные районы Москвы')
plt.xlabel('Названия районов')
plt.ylabel('Количество заведений')
plt.xticks(rotation=45)
plt.show()

Центр лидирует по колиству баров, кафе, кофейн и ресторанов, это и не удивительно туристическое место. Большой поток людей. В остальных более мене всё одинаково. Немного отличается Северо-Восточный округ. Самые маленькие показатели в Северо-западном округе. 

**Распределение средних рейтингов по категориям заведений.**

In [ ]:
mean_ratings = moscow_places.groupby('category')['rating'].mean().reset_index()
# создаем график
plt.figure(figsize=(12, 6))
sns.boxplot(x='category', y='rating', data=mean_ratings)
plt.title('Распределение средних рейтингов по категориям заведений')
plt.xlabel('Категория')
plt.ylabel('Средний рейтинг')
plt.xticks(rotation=45)
plt.show()

Самый высокий рейтинг у баров и пабов. Остальные +- одинаково от 4,20 до 4,30. Самый низкий у заведений быстрого питания, т.к. скорее всего вкус и качетсва продуктов неочень - 4,05 (шаурмичные)) и кафе 4,13. 

**Построим фоновую картограмму (хороплет) со средним рейтингом заведений каждого района.**

In [ ]:
rating_data = moscow_places.groupby('district', as_index=False)['rating'].agg('median')
ratings_geo = 'https://code.s3.yandex.net/datasets/admin_level_geomap.geojson'
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
Choropleth(
    geo_data=ratings_geo,
    data=rating_data,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Cредний рейтинг заведений каждого района',
).add_to(m)

m

Как и ожидалось в центре средняя оценка выше чем в остальных и держиться в районе 4.4. По большенству районов Москвы рейтинг держиться в районе 4,27 - 4,3. Юго - Восточный и Северо - Восточный административные округа и район Люберец отстают от общей оценки.

**Отобразить все заведения датасета на карте с помощью кластеров средствами библиотеки folium.**

In [ ]:
moscow_lat, moscow_lng = 55.751244, 37.618423
m = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(m)
def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']}",
    ).add_to(marker_cluster)

moscow_places.apply(create_clusters, axis=1)

m

Большая часть заведений находится в центре, что может быть связано с большим потоком туристов и жителей города в этом районе.

**Топ-15 улиц по количеству заведений.**

In [ ]:
# получаем топ-15 улиц по количеству заведений
streets = moscow_places['address'].str.split(',', expand=True)[1].str.strip()
top_15_streets = streets.value_counts().head(15).index.tolist()
top_15_streets = moscow_places[streets.isin(top_15_streets)]
grouped = top_15_streets.groupby(['street', 'category'])['name'].count().reset_index()

# построение графика
plt.figure(figsize=(20, 8))
sns.barplot(x='street', y='name', hue='category', data=grouped)
plt.title('Распределение количества заведений и их категорий по топ-15 улицам')
plt.xlabel('Улица')
plt.ylabel('Количество заведений')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Категория заведения', loc='upper right')
plt.show()

Проспект мира лидирует по кол-ву заведений, это связанно с тем что она длинная и находиться в туристическом районе москвы, рядом с ВДНХ.
На МКАДЕ анамальное количество кафе, связано с тем, что большое количество водителей останавливается перекусить,получается выгодно строить рядом с большой трассой кафе.

**Найдём улицы, на которых находится только один объект общепита.**

In [ ]:
street_counts = moscow_places['street'].value_counts()
one_place_streets = street_counts[street_counts == 1].index.tolist()

one_place = moscow_places[moscow_places['street'].isin(one_place_streets)]


grouped_one_place = one_place.groupby('street').mean().reset_index()
grouped_one_place = pd.merge(grouped_one_place, moscow_places[['street', 'lat', 'lng', 'name', 'category']], on='street', how='left')
grouped_one_place.drop_duplicates(subset='street', inplace=True)

moscow_lat, moscow_lng = 55.751244, 37.618423

a = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
# создаём пустой кластер, добавляем его на карту
marker_cluster = MarkerCluster().add_to(a)

def create_clusters(row):
    Marker(
        [row['lat_x'], row['lng_x']],
        popup=f"{row['name']} {row['rating']} {row['category']}",
    ).add_to(marker_cluster)

grouped_one_place.apply(create_clusters, axis=1)

a

Множество улиц с одним заведением находятся в центре, возможно это связано с тем, что в центре улицы небольшие и невыгодно держать большое количество заведений на одной улице. Плюс на этих улицах высокая конкуренция, что подтвергается риску.

Относительно заведений в других районах можно сказать, что наличие только одного заведения на улице может означать, что эта улица не является популярной для размещения общепита, и может не иметь высокого трафика посетителей.

In [ ]:
grouped_one_place['street'].nunique()

**Как удалённость от центра влияет на цены в заведениях.**

In [ ]:
rating_dengi = moscow_places.groupby('district')['middle_avg_bill'].median().reset_index()
moscow_lat, moscow_lng = 55.751244, 37.618423

d = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=ratings_geo,
    data=rating_dengi,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Цены',
).add_to(d)

d

Центральный и западный административные округа лидируют по среднему счету по Москве. Это связано с тем, что основная делавая активность проходит в этих округах.
Можно сделать вывод, что удаленность от центра города влияет на цены в заведениях. В более удаленных районах цены в среднем ниже, чем в центре города.

In [ ]:
moscow_places['middle_avg_bill'].describe()

In [ ]:
median_bill = moscow_places.groupby('district')['middle_avg_bill'].median().reset_index()
median_bill

**Общий вывод:**

**Категории заведений:**

- Москве наибольшее количество заведений общественного питания относится к категориям кафе, рестораны и кофейни.
- Большинство заведений являются несетевыми, однако сетевые заведения также имеют значительное присутствие на рынке.
- Лидерами среди сетевых заведений являются "Шоколадница", "Доминос Пицца" и "Додо Пицца".

**Кол-во мест:**

В заведениях, кроме булочных и пиццерий, максимальное количество посадочных мест обычно составляет 300, а среднее количество мест - около 150. Однако у булочных и пиццерий сетевых мест больше.

**Рейтинг:**

- Средняя оценка заведений в категории баров выше, чем в других категориях, что может свидетельствовать о более высоком качестве обслуживания и продуктов в барах.
- Заведения быстрого питания и кафе имеют более низкие оценки, что может быть связано с низким качеством продуктов и обслуживания.
- Большинство заведений имеют рейтинг от 4,21 до 4,3, что указывает на достаточно стабильный уровень качества обслуживания в общепите в Москве.
- Средняя оценка заведений в центре Москвы выше, чем в других районах, что может быть связано с более высоким уровнем конкуренции и требований к качеству обслуживания в центре города.

**Расположение:**

- В центральном административном округе Москвы больше всего заведений общественного питания, что объясняется его туристическим потенциалом и большим потоком людей.
- Наличие только одного заведения на улице в центре может быть связано с конкуренцией и невыгодностью держать большое количество заведений на одной улице. В других районах это может быть связано с низким спросом на общепит на данной улице.

**Цены:**

- Цены в заведениях в центре города выше, чем в более удаленных районах, возможно это связано с более высокой арендной платой и требованиями к качеству обслуживания в центре города.
- Средний чек центрального округа и западного самые высокие. Бюджетные заведения легче встретить ЮАО, ЮВАО и СВАО.

**Шаг 4. Детализируем исследование: открытие кофейни.**

- Сколько всего кофеен в датасете? В каких районах их больше всего, каковы особенности их расположения?
- Есть ли круглосуточные кофейни?
- Какие у кофеен рейтинги? Как они распределяются по районам?
- На какую стоимость чашки капучино стоит ориентироваться при открытии и почему?

In [ ]:
#узнаем сколько всего кофеен 
coffee_moscow = moscow_places[moscow_places['category'] == 'кофейня']
print(f"Всего кофеен: {len(coffee_moscow)}")

In [ ]:
coffee_district = coffee_moscow.groupby('district', as_index=False)['name'].agg('count').sort_values('name', ascending=False)

a = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=ratings_geo,
    data=coffee_district,
    columns=['district', 'name'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Количество кофеен по районам',
).add_to(a)

a

In [ ]:
# Количеством кофеен по районам
print("Количеством кофеен по районам:")
print(coffee_district)

Больше всего кофеен в ЦАО. Много туристических мест и также большое колиство офисов центров и других зданий с высоким колиством посетителей. ЮЗАО,ЮВАО и СЗАО находится меньше колиствов кофеен, т.к. являются спальными районами и имееют более низкую плотность населения и бизнеса. 

In [ ]:
#узнаем есть ли круглосуточные кофейни
coffee_times = coffee_moscow[coffee_moscow['is_24/7'] == True]
print(f"Круглосуточные кофейни: {len (coffee_times)}")

In [ ]:
coffee_times_distric = moscow_places[moscow_places['is_24/7'] == True]
n = Map(location=[moscow_lat, moscow_lng], zoom_start=10)
marker_cluster = MarkerCluster().add_to(n)

def create_clusters(row):
    Marker(
        [row['lat'], row['lng']],
        popup=f"{row['name']} {row['rating']} {row['category']}",
    ).add_to(marker_cluster)

coffee_times.apply(create_clusters, axis=1)

n

Всего 59 круглосуточных кофеен. Больше всего их в ЦАО.

In [ ]:
#район-медианное значение среднего чека
medians_coffee = coffee_moscow.groupby('district')['middle_avg_bill'].median().reset_index()
f = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=ratings_geo,
    data=medians_coffee,
    columns=['district', 'middle_avg_bill'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Средний чек по районам',
).add_to(f)

f

Тут мы посмотрели средний счёт в кофейнях. на первом месте ЗАО, затем следует ЦАО.

In [ ]:
rating_coffee = coffee_moscow.groupby('district', as_index=False)['rating'].agg('median')
w = Map(location=[moscow_lat, moscow_lng], zoom_start=10)

Choropleth(
    geo_data=ratings_geo,
    data=rating_coffee,
    columns=['district', 'rating'],
    key_on='feature.name',
    fill_color='YlGn',
    fill_opacity=0.8,
    legend_name='Рейтинг по районам',
).add_to(w)

w

Всё очень странно, лучших в средних ценах ЗАО - очень просел в рейтинге в отличии от других районов.

In [ ]:
#узнаем среднюю стоимость чашки в кофейне 
def average_cappuccino_price(dataframe):
    cappuccino_prices = []
    for feature in dataframe['avg_bill']:
        if isinstance(feature, str):
            match = re.search(r'Цена чашки капучино:\d+–(\d+)', feature)
            if match:
                cappuccino_prices.append(int(match.group(1)))

    avg_cappuccino_price = sum(cappuccino_prices) / len(cappuccino_prices)
    avg_cappuccino_price = round(avg_cappuccino_price, 2)
    return avg_cappuccino_price

In [ ]:
print('Средняя цена на чашку капучино равна:', average_cappuccino_price(moscow_places))

**Вывод
Рекомендации:**

- В западном административом округе средний счет самый высокий по Москве. Это может указывать на то, что в этом районе много людей, готовых потратить деньги на качественный кофе и дополнительные услуги. Но и самый низкий рейтинг, это говорит о то что не смотря, либо на плохое обслуживание или не вкусный кофе/выпечку люди всё равно покупают продукцию. И если открывать там кофейню и так сказать держать "марку", то возможно посетители перейдут к вам из-за сервиса и вкусного кофе.

- Учитывайте среднюю цену на чашку капучино в 216 рублей, чтобы определить ценовую политику вашей кофейни. Вам, возможно, стоит установить более конкурентоспособные цены, чтобы привлечь посетителей.

- Разработайте уникальную концепцию кофейни, чтобы выделиться на фоне конкурентов. Например, вы можете предложить редкие сорта кофе, уникальный дизайн интерьера или необычные десерты и закуски.

**Шаг 5. Подготовка презентации.**

Презентация: https://disk.yandex.ru/i/lbejgbwisQyPhQ